# Tech Intro

## Main tables

In [770]:
# Real Estate's objects
real_estate_list: list = [
    'ag_object',
    'ag_parse_ad',
    'ag_lead_object',
    'ag_presentation_object',
    'ag_object_platform',
    'ag_posting_ad_realty',
    'native_realty_filial',
    'foreign_realty_filial',
    'spr_metro',
    'spr_metro_connection',
    'ag_object_parking']
# Lead's objects
lead_list: list = [
    'leads',
    'lead_utm',
    'client_wishes',
    'leads_email_order',
    'request_from_site',
    'lead_finance',
    'leads_source_spr',
    'leads_calls_connection',
    'leads_pipeline',
    'leads_not_implemented_reason',
    'leads_status',
    'ag_pipeline',
    'ag_pipeline_status',
    'leads_meta',
    'leads_meta_fields',
    'leads_meta_group',
    'leads_contacts_connect',
    'leads_contact',
    'leads_contact_meta',
    'leads_contact_meta_fields']
# Deal's objects
deal_list: list = [
    # Deal
    'a_deal',
    'a_deal_meta',
    'deal_passport',
    'a_deal_finance',
    'a_deal_check',
    'a_deal_test',
    # Contractor
    'a_contractor',
    'a_contractor_meta',
    'a_deal_contractor',
    'a_deal_contractor_meta',
    # Object
    'a_object',
    'a_object_meta',
    'a_deal_object']

## Main Variables

In [771]:
# Address and name of your notebook
venv_path = "/home/bigdata/a.umarov/jupyter_P24"
# Exploratory Data Analysis(EDA) table (eda_table)
eda_table = deal_list[9] # You can generate full-EDA cycle if you want through the all of tables
                                # Just iterate it !
# Exploratory Data Analysis(EDA) obj as PySpark DataFrame
eda_df = None
# Set the date of checking
sample_data_date = '20221027'

## Useful command and modules (can be transform into the 'importlib'- smart import logic)

In [772]:
import os
import pandas as pd
# To data analyze need the module
import pyspark.pandas as ps
os.environ["PYSPARK_PYTHON"] = f"{venv_path}/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = f"{venv_path}/bin/python3"

import findspark
findspark.init("/opt/dev/spark/spark")
 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, DataFrameNaFunctions, \
                        Window, DataFrame, DataFrameStatFunctions, \
                        Column, Row, GroupedData, functions, types
from pyspark.sql.functions import isnan, when, count, col


conf = SparkConf().setAppName("get_info_from_" + eda_table + "_table")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
spark = SparkSession(sc)

W1120 02:18:36.557492 10390 process.cpp:2877] Attempted to spawn already running process version@10.62.4.11:44708
I1120 02:18:36.558382 38323 sched.cpp:241] Version: 1.11.0
I1120 02:18:36.558701 10570 sched.cpp:345] New master detected at master@10.62.4.8:5050
I1120 02:18:36.558780 10570 sched.cpp:365] No credentials provided. Attempting to register without authentication
I1120 02:18:36.560081 10554 sched.cpp:753] Framework registered with 89161c75-8817-4ae4-ab24-cba26241d6f9-75624


## Read the spec-file from Hadoop and show some data sample

In [773]:
eda_df = spark.read.parquet("/prod/source/p24/" + eda_table + "/datekey=" + sample_data_date)
eda_df.show(10, False)

+-------+---------------+-------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|id     |deal_contractor|key                                                    |value                                                                                                                                                                               |updated             |
+-------+---------------+-------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|1000005|176201         |Документ, на основании которого действует представитель|                                                                   

## Show columns' properties

In [774]:
print('Data overview')
eda_df.printSchema()

Data overview
root
 |-- id: string (nullable = true)
 |-- deal_contractor: string (nullable = true)
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)
 |-- updated: string (nullable = true)



## Total rows (count)

In [775]:
eda_df.count()

1103153

## Simple func to execute and show the result of the SQL-instruction

In [776]:
def execute(select_instruction: str = "select count(*) from eda_sql") -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "select count(*) from eda_sql"
    :param :string select_instruction: SQL statement
    :return: result of the SQL-operation as a table
    """
    return spark.sql(select_instruction).show(truncate = False)

## Create the temp-table-view from the DF (called as "eda_sql")

In [777]:
eda_df.createOrReplaceTempView("eda_sql")

## Check the method and temp-table

In [778]:
execute("select count(*) from eda_sql")

+--------+
|count(1)|
+--------+
|1103153 |
+--------+



## Detect missing values and abnormal zeroes and print it

###### Check-rule Matrix


|  Name of the rule    |Available                   
|----------------------|-------------------------|
|NULL/NaN/NA           |+                        |
|'Zero'-value          |+                        |
|'Empty-string'        |+                        |

### 1st solution (PySpark & Pandas version)
#### Max perfomance but harder to read & understand

##### Filter and count after - all the values in the  our EDA-DF

In [779]:
# NULL/NaN/NA values condition
eda_df_null_values = eda_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_null_values

,id,deal_contractor,key,value,updated
0,0,0,0,37,0


In [780]:
# Zero values condition
eda_df_zero_values = eda_df.select([count(when(col(c) == 0, c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_zero_values

,id,deal_contractor,key,value,updated
0,0,0,0,0,0


In [781]:
# Empty-string values condition
eda_df_empty_values = eda_df.select([count(when(col(c) == '', c)).alias(c) \
                                     for c in eda_df.columns]).toPandas()
eda_df_empty_values

,id,deal_contractor,key,value,updated
0,0,0,0,698897,0


In [782]:
# Unlock the limit to display all the columns
pd.options.display.max_rows = None
# To transpose a dataframe - use .T method
eda_df_pandas = pd.concat([eda_df_null_values, eda_df_zero_values, eda_df_empty_values], axis = 'index').T
# To rename columns to anything else after the transpose
eda_df_pandas.columns = (['NULL','Zero','Empty_string'])
eda_df_pandas

,NULL,Zero,Empty_string
id,0,0,0
deal_contractor,0,0,0
key,0,0,0
value,37,0,698897
updated,0,0,0


In [783]:
def set_result_to_percent(df: object = None) -> object:
    """
    Method to execute and show the result of your PandasDataFrame
    :default -> None-object
    :param :Pandas DF
    :return: DF after the conversion (absolute -> percent)
    """
    if df is None:
        return "Please incl. at least one Pandas DF as a parameter"
    return round(df * 100 / eda_df.count(), 3)

In [784]:
set_result_to_percent(eda_df_pandas)

,NULL,Zero,Empty_string
id,0.000,0.0,0.000
deal_contractor,0.000,0.0,0.000
key,0.000,0.0,0.000
value,0.003,0.0,63.354
updated,0.000,0.0,0.000


### SQL-instructions

In [785]:
#Find dupes or misunderstanding or useless static values 3 244 006 total
for col in eda_df.columns:
    execute("select count(distinct("+ col + ")) from eda_sql")

+------------------+
|count(DISTINCT id)|
+------------------+
|1103153           |
+------------------+



+-------------------------------+
|count(DISTINCT deal_contractor)|
+-------------------------------+
|367613                         |
+-------------------------------+

+-------------------+
|count(DISTINCT key)|
+-------------------+
|3                  |
+-------------------+

+---------------------+
|count(DISTINCT value)|
+---------------------+
|30001                |
+---------------------+

+-----------------------+
|count(DISTINCT updated)|
+-----------------------+
|303486                 |
+-----------------------+



In [786]:
def execute_template(column: str ='id') -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "ID incl."
    :param :string select_instruction: column name
    :return: result of the SQL-operation as a table
    """
    print(execute("SELECT DISTINCT(" + column + ")\
                FROM eda_sql\
                WHERE 1 = 1 \
                ORDER BY " + column + " DESC"))
    return execute("SELECT DISTINCT(" + column + ")\
                    FROM eda_sql\
                    WHERE 1 = 1 \
                    ORDER BY " + column + " ASC")

In [787]:
#Find dupes or misunderstanding or useless static values 3 244 006 total
for col in eda_df.columns:
    execute_template(col)

+------+
|id    |
+------+
|999999|
|999998|
|999997|
|999990|
|99999 |
|999989|
|999988|
|999987|
|999986|
|999985|
|999984|
|999983|
|999982|
|999981|
|999980|
|99998 |
|999979|
|999978|
|999977|
|999976|
+------+
only showing top 20 rows

None
+-------+
|id     |
+-------+
|10000  |
|100000 |
|1000000|
|1000001|
|1000002|
|1000003|
|1000004|
|1000005|
|1000006|
|1000007|
|1000008|
|1000009|
|100001 |
|1000010|
|1000011|
|1000015|
|1000016|
|1000017|
|100002 |
|1000021|
+-------+
only showing top 20 rows

+---------------+
|deal_contractor|
+---------------+
|99999          |
|99998          |
|99997          |
|99996          |
|99995          |
|99994          |
|99993          |
|99992          |
|99991          |
|9999           |
|99989          |
|99988          |
|99987          |
|99986          |
|99985          |
|99984          |
|99983          |
|99981          |
|99980          |
|99979          |
+---------------+
only showing top 20 rows

None
+---------------+
|deal_

## Bonus

### 2nd solution (Cycle and variables)
#### Slower but more simple and has more readability than the previous method 

### Create the list of variables to re-use if needed

In [ ]:
useless_columns = []
null_values = []
zero_values = []
empty_string = []
col_name = []

In [ ]:
for col in eda_df.columns:
    null = round((eda_df.filter(eda_df[col].isNull()).count()/eda_df.count())*100, 3)
    zero = round(eda_df.filter(eda_df[col]==0).count()/eda_df.count()*100, 3)
    empty = round(eda_df.filter(eda_df[col]=='').count()/eda_df.count()*100, 3)
    if null in [0, 100] and zero in [0, 100] and empty in [0, 100]:
        useless_columns.append(col)
        continue
    null_values.append(null)
    zero_values.append(zero)
    empty_string.append(empty)
    col_name.append(col)

In [ ]:
zipped = list(zip(null_values, zero_values, empty_string))
eda_df_pandas2 = pd.DataFrame(zipped, index=col_name, columns=['NULL', 'Zero', 'Empty'])
eda_df_pandas2

### Remove useless columns

In [ ]:
eda_df = eda_df.drop(*useless_columns)

In [ ]:
pd.DataFrame(eda_df.dtypes, columns = ['Column Name','Data type'])

In [769]:
spark.stop()

I1120 02:18:32.591466 10390 sched.cpp:2174] Asked to stop the driver
I1120 02:18:32.591548 10587 sched.cpp:1207] Stopping framework 89161c75-8817-4ae4-ab24-cba26241d6f9-75622
